In [1]:
from dataclasses import dataclass, field
import functools
import itertools
import re

from pyprojroot import here

In [8]:
@dataclass
class Volume:
    cubes: set[tuple[int, int, int]]
    grid: set[tuple[int, int, int]] = field(init=False)
    maxX: int = field(init=False)
    maxY: int = field(init=False)
    maxZ: int = field(init=False)

    def __post_init__(self):
        self.translateToOrigin()
        self.maxX = max(x for x, y, z in self.cubes)
        self.maxY = max(y for x, y, z in self.cubes)
        self.maxZ = max(z for x, y, z in self.cubes)
        self.grid = set(itertools.product(range(self.maxX + 1), range(self.maxY + 1), range(self.maxZ + 1)))

    def translateToOrigin(self):
        minX = min(x for x, y, z in self.cubes)
        minY = min(y for x, y, z in self.cubes)
        minZ = min(z for x, y, z in self.cubes)
        self.cubes = set((x - minX, y - minY, z - minZ) for x, y, z in self.cubes)

    def cubeSurfaceArea(self, cube: tuple[int, int, int]):
        x, y, z = cube
        adjacentCubes = [(x, y, z - 1),(x, y, z + 1), (x, y - 1, z), (x, y + 1, z),(x - 1, y, z), (x + 1, y, z)]
        return sum([cube not in self.cubes for cube in adjacentCubes])

    def surfaceArea(self) -> int:
        return sum(map(self.cubeSurfaceArea, self.cubes))

In [6]:
@dataclass
class Boulder(Volume):

    def findSurroundingAir(self) -> set[tuple[int, int, int]]:
        grid = set(itertools.product(range(self.maxX + 1), range(self.maxY + 1), range(self.maxZ + 1)))
        surroundingAir = set()

        # sort and groupby keys
        xyKey = lambda coords: (coords[0], coords[1])
        xzKey = lambda coords: (coords[0], coords[2])
        yzKey = lambda coords: (coords[1], coords[2])

        # groupby xy, air columns along z axis
        for key, group in itertools.groupby(sorted(self.cubes, key=xyKey), key=xyKey):
            x, y = key
            group = list(group)
            minZ = min(z for x, y, z in group)
            maxZ = max(z for x, y, z in group)
            airColumn = set((x, y, z) for z in itertools.chain(range(minZ), range(maxZ + 1, self.maxZ + 1)))
            surroundingAir.update(airColumn)

        # groupby xz, air columns along y axis
        for key, group in itertools.groupby(sorted(self.cubes, key=xzKey), key=xzKey):
            x, z = key
            group = list(group)
            minY = min(y for x, y, z in group)
            maxY = max(y for x, y, z in group)
            airColumn = set((x, y, z) for y in itertools.chain(range(minY), range(maxY + 1, self.maxY + 1)))
            surroundingAir.update(airColumn)

        # groupby yz, air columns along x axis
        for key, group in itertools.groupby(sorted(self.cubes, key=yzKey), key=yzKey):
            y, z = key
            group = list(group)
            minX = min(x for x, y, z in group)
            maxX = max(x for x, y, z in group)
            airColumn = set((x, y, z) for x in itertools.chain(range(minX), range(maxX + 1, self.maxX + 1)))
            surroundingAir.update(airColumn)

        return surroundingAir

    def findAirPockets(self) -> Volume:
        return Volume(self.grid - self.findSurroundingAir() - self.cubes)

In [10]:
testPath = here('./18/test.txt')
path = here('./18/input.txt')

with open(testPath, 'r') as fp:
    lines = fp.readlines()
    cubes = set([tuple(int(char) for char in (re.findall('\d+', line))) for line in lines])
    boulder = Boulder(cubes)

print(len(boulder.grid))
print(len(boulder.cubes))
print(len(boulder.findSurroundingAir()))
print(len(boulder.findAirPockets().cubes))

54
13


ValueError: min() arg is an empty sequence

In [28]:
gridCubes = set(itertools.product(range(1, 4), range(1, 4), range(1, 4)))
boulderCubes = (gridCubes - set(itertools.product((1,3), repeat=3)) - set([(2, 2, 2)]))
boulder = Boulder(boulderCubes)
airPockets = boulder.findAirPockets()
surfaceArea = boulder.surfaceArea() - airPockets.surfaceArea()
surfaceArea

-480

In [12]:
boulder.surfaceArea()

60

In [13]:
airPockets.surfaceArea()

6